In [ ]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)

In [ ]:
import sys
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle as cp
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import datetime as dt
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

###  Hyperparameters

In [ ]:
hidden_units = 200
epochs = 100
lookback = 7
learning_rate = 0.001
batch_size = 1024
regularizer = 'l2'
dropouts = {'dropout':0.55,'recurrent_dropout':0.3}

crops = ['Onion','Potato','Wheat','Rice']
crop = crops[0]

train_split = 2557 - 1 #if using price diff

fileformat = 'food_price_data/%s.csv'

### Model

In [ ]:
def ren_train(X, y, hidden_units=100, learning_rate=0.01, epochs=200, batch_size=256,dropouts={'dropout':0.25,'recurrent_dropout':0.25},regularizer='l2'):
    model = Sequential()
    model.add(LSTM(hidden_units, activation='relu', recurrent_regularizer=regularizer, activity_regularizer=regularizer, 
                   dropout=dropouts['dropout'], recurrent_dropout=dropouts['recurrent_dropout'], input_shape=(X.shape[1], X.shape[-1])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X, y, epochs=epochs, verbose=1)
    plt.plot(history.history['loss'])
    plt.show()
    return model

### Data

In [ ]:
def toPDTimeSeries(crop,start,end):
    startdate = dt.datetime.strptime(start, '%Y-%m-%d').date()
    enddate = dt.datetime.strptime(end, '%Y-%m-%d').date()
    data = pd.read_csv(fileformat%(crop))
    print(data.head())
    price = data['price']
    dates = data['date']
    
    dateindex = pd.DatetimeIndex(dates)
    print(dateindex[0]+31)
    print (dateindex)
    price_data = pd.Series(price)#, index=dates)#index)
    return price_data[start:end]
    

In [ ]:
def addSpanLag(data, events=[], look_back=7, look_ahead = 7):##span==1
#     ### Adding span (required for weekly, monthly average data)
#     if span>1:
#         nts = []
#         ind = 0
#         while ind < len(ts):
#             tmpts = ts[ind:ind+span]
#             print (tmpts)
#             dt = tmpts[0]
#             vals = [k for k in tmpts]
#             pr = sum(vals)/len(vals)
#             nts.append((dt,pr))
#             ind+=span
#         ts = nts
#     if not convert_to_Xy:
#         return ts

    ### Adding lag (based on look back and look ahead)
    
    if len(events) > 0:
        input = np.append(data,events,axis=1)
    else:
        input = data
    #     season = np.array([k%365 for k in range(train_split)])
    #     season = season.reshape(len(season),1)
    #     input = np.append(input,season,axis=1)
    ind = 0
    X_data = []
    y_data = []

    while ind + look_back + look_ahead < len(data):
        
        X = input[ind:ind+look_back]
        Y = data[ind+look_back+look_ahead]
        X_data.append(X)
        y_data.append(Y)
        ind+=1
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data

#### Reading food prices

In [ ]:
food_price = pd.read_csv(fileformat%(crop))['price']
food_price_diff = food_price.diff().dropna() 
price_diff = np.array(food_price_diff).reshape(food_price_diff.shape[0],1)


#### Reading embeddings

In [ ]:
### Events
events_train = np.load('embeddings/events_train.npy')[1:]
events_test = np.load('embeddings/events_test.npy')
events = np.append(events_train,events_test, axis=0)

### Topics
topics_train = np.load('embeddings/toi.LDA_train.npy')[1:]
topic_test = np.load('embeddings/toi.LDA_test.npy')
topics = np.append(topics_train,topic_test, axis=0)

### Word2Vec
w2v_train = np.load('embeddings/toi.W2V_train.npy')[1:]
w2v_test = np.load('embeddings/toi.W2V_test.npy')
w2v = np.append(w2v_train,w2v_test, axis=0)

### Rearranging and splitting the data

In [ ]:
X_event, y_event   = addSpanLag(price_diff,events,look_back=7,look_ahead=0)
X_event_train = X_event[:train_split]
y_event_train = y_event[:train_split]
X_event_test = X_event[train_split:]
y_event_test = y_event[train_split:]

X_topics, y_topics   = addSpanLag(price_diff,topics,look_back=7,look_ahead=0)
X_topics_train = X_topics[:train_split]
y_topics_train = y_topics[:train_split]
X_topics_test = X_topics[train_split:]
y_topics_test = y_topics[train_split:]

X_w2v, y_w2v   = addSpanLag(price_diff,w2v,look_back=7,look_ahead=0)
X_w2v_train = X_w2v[:train_split]
y_w2v_train = y_w2v[:train_split]
X_w2v_test = X_w2v[train_split:]
y_w2v_test = y_w2v[train_split:]


X_noevent, y_noevent   = addSpanLag(price_diff,look_back=7,look_ahead=0)
X_noevent_train = X_noevent[:train_split]
y_noevent_train = y_noevent[:train_split]
X_noevent_test = X_noevent[train_split:]
y_noevent_test = y_noevent[train_split:]

In [ ]:
print(X_event_train.shape)
print(y_event_train.shape)
print(X_event_test.shape)
print(y_event_test.shape)

print(X_topics_train.shape)
print(y_topics_train.shape)
print(X_topics_test.shape)
print(y_topics_test.shape)

print(X_w2v_train.shape)
print(y_w2v_train.shape)
print(X_w2v_test.shape)
print(y_w2v_test.shape)


print(X_noevent_train.shape)
print(y_noevent_train.shape)
print(X_noevent_test.shape)
print(y_noevent_test.shape)

### Training

In [ ]:
### REN
model_event = ren_train(X_event_train,y_event_train,hidden_units=hidden_units,learning_rate=learning_rate,
                        epochs=epochs,dropouts=dropouts,regularizer=regularizer)

### LSTM + LDA
model_topics = ren_train(X_topics_train,y_topics_train,hidden_units=hidden_units,learning_rate=learning_rate,
                         epochs=epochs,dropouts=dropouts,regularizer=regularizer)

### LSTM + W2V
model_w2v = ren_train(X_w2v_train,y_w2v_train,hidden_units=hidden_units,learning_rate=learning_rate,
                      epochs=epochs,dropouts=dropouts,regularizer=regularizer)

### LSTM
model_noevent = ren_train(X_noevent_train,y_noevent_train,hidden_units=hidden_units,learning_rate=learning_rate,
                          epochs=epochs,dropouts=dropouts,regularizer=regularizer)


In [ ]:
model_event.save('models/ren')
model_topics.save('models/lda')
model_w2v.save('models/w2v')
model_noevent.save('models/lstm')

### Testing

#### Computing model performance

In [ ]:
print('REN\n===============')
tr_acc = model_event.evaluate(X_event_train,y_event_train)
tst_acc_13_14 = model_event.evaluate(X_event_test[:730],y_event_test[:730])
print("training accuracy: %.3f\ntest accuracy 2013-14: %.3f"%(np.sqrt(tr_acc),np.sqrt(tst_acc_13_14)))
print()

print('LDA\n===============')
tr_acc = model_topics.evaluate(X_topics_train,y_topics_train)
tst_acc_13_14 = model_topics.evaluate(X_topics_test[:730],y_topics_test[:730])
print("training accuracy: %.3f\ntest accuracy 2013-14: %.3f"%(np.sqrt(tr_acc),np.sqrt(tst_acc_13_14)))
print()

print('W2V\n===============')
tr_acc = model_w2v.evaluate(X_w2v_train,y_w2v_train)
tst_acc_13_14 = model_w2v.evaluate(X_w2v_test[:730],y_w2v_test[:730])
print("training accuracy: %.3f\ntest accuracy 2013-14: %.3f"%(np.sqrt(tr_acc),np.sqrt(tst_acc_13_14)))
print()

print('LSTM\n===============')
tr_acc = model_noevent.evaluate(X_noevent_train,y_noevent_train)
tst_acc_13_14 = model_noevent.evaluate(X_noevent_test[:730],y_noevent_test[:730])
yhat=model_noevent.predict(X_noevent_test)
tst_acc = mean_squared_error(yhat,y_noevent_test,squared=False)
print("training accuracy: %.3f\ntest accuracy: %.3f\ntest accuracy 2013-14: %.3f"%(np.sqrt(tr_acc),tst_acc,np.sqrt(tst_acc_13_14)))


In [ ]:
### test